In [18]:
import pandas as pd
from datetime import datetime

### Load Divvy and Weather Data

In [2]:
%time
divvy_data = pd.read_csv("~/dev/me/chipy-mentorship/divvy.csv")

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.11 µs


/Users/jdaubner/.envs/divy/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
divvy_data.shape

(9527208, 20)

In [11]:
divvy_data.head()
divvy_data.columns

Index(['Unnamed: 0', 'trip_id', 'starttime', 'stoptime', 'bikeid',
       'tripduration', 'from_station_id', 'from_station_name', 'to_station_id',
       'to_station_name', 'usertype', 'gender', 'birthyear', 'DATE', 'PRCP',
       'SNOW', 'SNWD', 'TAVG', 'TMAX', 'TMIN'],
      dtype='object')

### Drop Unnecessary and Empty Weather Data Columns

In [13]:
bad_columns = ['Unnamed: 0', 'PRCP', 'SNOW', 'SNWD',
               'TAVG', 'TMAX', 'TMIN', 'DATE']
divvy_data_cleanned = divvy_data.drop(labels=bad_columns,axis=1)
divvy_data_cleanned.head()

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
0,9379901,2016-04-30 23:59:00,2016-05-01 00:11:00,21,733,123,California Ave & Milwaukee Ave,374,Western Ave & Walton St,Subscriber,Male,1982.0
1,9379900,2016-04-30 23:58:00,2016-05-01 00:07:00,3583,556,349,Halsted St & Wrightwood Ave,165,Clark St & Grace St,Subscriber,Male,1991.0
2,9379899,2016-04-30 23:58:00,2016-05-01 00:02:00,4557,253,59,Wabash Ave & Roosevelt Rd,273,Michigan Ave & 18th St,Subscriber,Male,1984.0
3,9379898,2016-04-30 23:54:00,2016-05-01 00:08:00,2443,802,289,Wells St & Concord Ln,199,Wabash Ave & Grand Ave,Subscriber,Male,1978.0
4,9379897,2016-04-30 23:52:00,2016-05-01 00:11:00,50,1146,239,Western Ave & Leland Ave,227,Southport Ave & Waveland Ave,Customer,NaN,NaN


In [5]:
weather_data = pd.read_csv("~/dev/me/chipy-mentorship/weather_data.csv")

In [6]:
weather_data.shape

(1094, 10)

In [7]:
weather_data.head()

,STATION,NAME,DATE,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WDMV
0,USC00111497,"CHICAGO BOTANIC GARDEN, IL US",1/1/2016,0.0,0.0,3.0,30.0,18.0,23.0,NaN
1,USC00111497,"CHICAGO BOTANIC GARDEN, IL US",1/2/2016,0.0,0.0,3.0,30.0,21.0,23.0,NaN
2,USC00111497,"CHICAGO BOTANIC GARDEN, IL US",1/3/2016,0.0,0.0,3.0,36.0,22.0,28.0,NaN
3,USC00111497,"CHICAGO BOTANIC GARDEN, IL US",1/4/2016,0.0,0.0,3.0,30.0,24.0,26.0,NaN
4,USC00111497,"CHICAGO BOTANIC GARDEN, IL US",1/5/2016,0.0,0.0,3.0,32.0,19.0,22.0,NaN


### Use the Same Data Format in Each DataFrame
This is necessary for us to be able to join the two DataFrames on date.

In [27]:
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')
divvy_data_cleanned['start_date'] = pd.to_datetime(divvy_data_cleanned['starttime'], format='%Y-%m-%d').dt.date

In [29]:
divvy_data_cleanned[['starttime', 'start_date']].head()

,starttime,start_date
0,2016-04-30 23:59:00,2016-04-30
1,2016-04-30 23:58:00,2016-04-30
2,2016-04-30 23:58:00,2016-04-30
3,2016-04-30 23:54:00,2016-04-30
4,2016-04-30 23:52:00,2016-04-30


In [31]:
weather_data['date'] = pd.to_datetime(weather_data['DATE'], format='%m/%d/%Y').dt.date
weather_data[['DATE', 'date']].head()

,DATE,date
0,1/1/2016,2016-01-01
1,1/2/2016,2016-01-02
2,1/3/2016,2016-01-03
3,1/4/2016,2016-01-04
4,1/5/2016,2016-01-05


### Join Weather and Divvy Data by Date

In [32]:
divvy_plus_weather_data = divvy_data_cleanned.merge(weather_data,
                                                    how='left',
                                                    left_on='start_date',
                                                    right_on='date')
divvy_plus_weather_data.head()

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,...,NAME,DATE,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WDMV,date
0,9379901,2016-04-30 23:59:00,2016-05-01 00:11:00,21,733,123,California Ave & Milwaukee Ave,374,Western Ave & Walton St,Subscriber,...,"CHICAGO BOTANIC GARDEN, IL US",4/30/2016,0.0,0.0,0.0,49.0,41.0,45.0,NaN,2016-04-30
1,9379900,2016-04-30 23:58:00,2016-05-01 00:07:00,3583,556,349,Halsted St & Wrightwood Ave,165,Clark St & Grace St,Subscriber,...,"CHICAGO BOTANIC GARDEN, IL US",4/30/2016,0.0,0.0,0.0,49.0,41.0,45.0,NaN,2016-04-30
2,9379899,2016-04-30 23:58:00,2016-05-01 00:02:00,4557,253,59,Wabash Ave & Roosevelt Rd,273,Michigan Ave & 18th St,Subscriber,...,"CHICAGO BOTANIC GARDEN, IL US",4/30/2016,0.0,0.0,0.0,49.0,41.0,45.0,NaN,2016-04-30
3,9379898,2016-04-30 23:54:00,2016-05-01 00:08:00,2443,802,289,Wells St & Concord Ln,199,Wabash Ave & Grand Ave,Subscriber,...,"CHICAGO BOTANIC GARDEN, IL US",4/30/2016,0.0,0.0,0.0,49.0,41.0,45.0,NaN,2016-04-30
4,9379897,2016-04-30 23:52:00,2016-05-01 00:11:00,50,1146,239,Western Ave & Leland Ave,227,Southport Ave & Waveland Ave,Customer,...,"CHICAGO BOTANIC GARDEN, IL US",4/30/2016,0.0,0.0,0.0,49.0,41.0,45.0,NaN,2016-04-30


In [33]:
divvy_plus_weather_data.shape

(9527208, 24)

In [34]:
divvy_plus_weather_data[['PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'TOBS', 'WDMV']].describe()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WDMV
count,9.498366e+06,8.073564e+06,8.045113e+06,9.494047e+06,9.502661e+06,9.502661e+06,6.580197e+06
mean,1.187134e-01,2.456002e-02,1.388659e-01,6.848671e+01,4.932205e+01,5.614396e+01,2.167987e+01
std,3.442404e-01,2.572655e-01,8.252898e-01,1.776356e+01,1.567575e+01,1.582938e+01,1.716720e+02
min,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00,-8.000000e+00,-6.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,5.500000e+01,3.800000e+01,4.400000e+01,5.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,7.300000e+01,5.300000e+01,6.100000e+01,9.900000e+00
75%,5.000000e-02,0.000000e+00,0.000000e+00,8.200000e+01,6.300000e+01,6.900000e+01,1.990000e+01
max,3.760000e+00,6.400000e+00,1.300000e+01,9.700000e+01,7.800000e+01,8.300000e+01,4.460800e+03
